In [16]:
import pandas as pd 
df = pd.read_csv('cleaned_sales.csv')

In [17]:
df.sample(20)

,sale_date,domain,quantity
50944,2023-05-25,Electronics,51
27111,2022-09-29,Toys,257
43628,2023-03-13,Clothing,127
1888,2022-01-19,Toys,170
9819,2022-04-09,Furniture,302
60945,2023-09-02,Electronics,8
51756,2023-06-02,Toys,10
66023,2023-10-23,Groceries,5
72159,2023-12-23,Groceries,221
36926,2023-01-05,Clothing,33


In [18]:
df["sale_date"] = pd.to_datetime(df["sale_date"])
df.head()


,sale_date,domain,quantity
0,2022-01-01,Groceries,127
1,2022-01-01,Toys,150
2,2022-01-01,Toys,65
3,2022-01-01,Toys,61
4,2022-01-01,Electronics,14


In [19]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 73100 entries, 0 to 73099
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   sale_date  73100 non-null  datetime64[ns]
 1   domain     73100 non-null  object        
 2   quantity   73100 non-null  int64         
dtypes: datetime64[ns](1), int64(1), object(1)
memory usage: 1.7+ MB


In [110]:
weekly_df = (df.set_index("sale_date").groupby("domain")["quantity"].resample("W-MON").sum().reset_index())

In [111]:
weekly_df.sample(5)

,domain,sale_date,quantity
158,Electronics,2023-01-09,17789
251,Furniture,2022-10-17,22148
368,Groceries,2023-01-09,19901
87,Clothing,2023-09-04,16932
375,Groceries,2023-02-27,19655


In [112]:
weekly_df = weekly_df.rename(columns={"quantity": "weekly_quantity"})
weekly_df.sample(6)


,domain,sale_date,weekly_quantity
0,Clothing,2022-01-03,8634
522,Toys,2023-12-18,17618
497,Toys,2023-06-26,18899
262,Furniture,2023-01-02,17477
154,Electronics,2022-12-12,16165
161,Electronics,2023-01-30,18630


In [113]:
weekly_df["lag_1"] = weekly_df.groupby("domain")["weekly_quantity"].shift(1)
weekly_df["lag_2"] = weekly_df.groupby("domain")["weekly_quantity"].shift(2)
weekly_df["lag_4"] = weekly_df.groupby("domain")["weekly_quantity"].shift(4)
weekly_df["lag_8"] = weekly_df.groupby("domain")["weekly_quantity"].shift(8)

In [114]:
weekly_df.sample(5)

,domain,sale_date,weekly_quantity,lag_1,lag_2,lag_4,lag_8
223,Furniture,2022-04-04,21859,21083.0,17929.0,18648.0,17372.0
43,Clothing,2022-10-31,16249,18092.0,16874.0,20579.0,21083.0
125,Electronics,2022-05-23,18923,16894.0,17643.0,18072.0,20535.0
523,Toys,2023-12-25,18029,17618.0,16980.0,21671.0,20477.0
425,Toys,2022-02-07,20478,16740.0,17508.0,19992.0,NaN


In [115]:
weekly_df.head()
weekly_df["target_of_week"] = weekly_df.groupby("domain")["weekly_quantity"].shift(-1)


In [116]:
weekly_df.head()

,domain,sale_date,weekly_quantity,lag_1,lag_2,lag_4,lag_8,target_of_week
0,Clothing,2022-01-03,8634,NaN,NaN,NaN,NaN,17757.0
1,Clothing,2022-01-10,17757,8634.0,NaN,NaN,NaN,17896.0
2,Clothing,2022-01-17,17896,17757.0,8634.0,NaN,NaN,16235.0
3,Clothing,2022-01-24,16235,17896.0,17757.0,NaN,NaN,16955.0
4,Clothing,2022-01-31,16955,16235.0,17896.0,8634.0,NaN,20282.0


In [117]:
weekly_df["roll_4"] = (
    weekly_df.groupby("domain")["weekly_quantity"]
    .shift(1)
    .rolling(4)
    .mean()
)

weekly_df["roll_8"] = (
    weekly_df.groupby("domain")["weekly_quantity"]
    .shift(1)
    .rolling(8)
    .mean()
)


In [118]:
model_df = weekly_df.dropna().reset_index(drop=True)

In [119]:
model_df.shape

(480, 10)

In [120]:
model_df['domain'].unique()

array(['Clothing', 'Electronics', 'Furniture', 'Groceries', 'Toys'],
      dtype=object)

In [121]:
model_df.sample(5)

,domain,sale_date,weekly_quantity,lag_1,lag_2,lag_4,lag_8,target_of_week,roll_4,roll_8
134,Electronics,2022-11-21,16136,19315.0,20339.0,19448.0,17472.0,18868.0,19684.75,19634.250
449,Toys,2023-05-29,19495,16657.0,18234.0,19808.0,17918.0,16007.0,18904.00,19154.375
187,Electronics,2023-11-27,19812,19742.0,16249.0,23004.0,19152.0,19358.0,18976.25,19272.750
40,Clothing,2022-12-05,23291,22858.0,17381.0,18690.0,20321.0,20692.0,20057.25,18970.625
259,Furniture,2023-06-12,21712,16690.0,15433.0,18799.0,16617.0,19748.0,17015.25,17635.500


In [122]:

X = model_df[["domain", "lag_1", "lag_2", "lag_4", "lag_8", "roll_4", "roll_8"]]
y = model_df["target_of_week"]


In [123]:

from sklearn.preprocessing import OrdinalEncoder
from sklearn.compose import ColumnTransformer

preprocessor = ColumnTransformer(
    transformers=[
        ("domain_enc", OrdinalEncoder(), ["domain"])
    ],
    remainder="passthrough"
)


In [124]:

from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestRegressor

model = Pipeline(
    steps=[
        ("preprocess", preprocessor),
        ("model", RandomForestRegressor(
            n_estimators=200,
            random_state=42
        ))
    ]
)


In [125]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

model.fit(X_train, y_train)


,"steps steps: list of tuplesList of (name of step, estimator) tuples that are to be chained insequential order. To be compatible with the scikit-learn API, all stepsmust define `fit`. All non-last steps must also define `transform`. See:ref:`Combining Estimators ` for more details.","[('preprocess', ...), ('model', ...)]"
,"transform_input transform_input: list of str, default=NoneThe names of the :term:`metadata` parameters that should be transformed by thepipeline before passing it to the step consuming it.This enables transforming some input arguments to ``fit`` (other than ``X``)to be transformed by the steps of the pipeline up to the step which requiresthem. Requirement is defined via :ref:`metadata routing `.For instance, this can be used to pass a validation set through the pipeline.You can only set this if metadata routing is enabled, which youcan enable using ``sklearn.set_config(enable_metadata_routing=True)``... versionadded:: 1.6",None
,"memory memory: str or object with the joblib.Memory interface, default=NoneUsed to cache the fitted transformers of the pipeline. The last stepwill never be cached, even if it is a transformer. By default, nocaching is performed. If a string is given, it is the path to thecaching directory. Enabling caching triggers a clone of the transformersbefore fitting. Therefore, the transformer instance given to thepipeline cannot be inspected directly. Use the attribute ``named_steps``or ``steps`` to inspect estimators within the pipeline. Caching thetransformers is advantageous when fitting is time consuming. See:ref:`sphx_glr_auto_examples_neighbors_plot_caching_nearest_neighbors.py`for an example on how to enable caching.",None
,"verbose verbose: bool, default=FalseIf True, the time elapsed while fitting each step will be printed as itis completed.",False
,"transformers transformers: list of tuplesList of (name, transformer, columns) tuples specifying thetransformer objects to be applied to subsets of the data.name : str Like in Pipeline and FeatureUnion, this allows the transformer and its parameters to be set using ``set_params`` and searched in grid search.transformer : {'drop', 'passthrough'} or estimator Estimator must support :term:`fit` and :term:`transform`. Special-cased strings 'drop' and 'passthrough' are accepted as well, to indicate to drop the columns or to pass them through untransformed, respectively.columns : str, array-like of str, int, array-like of int, array-like of bool, slice or callable Indexes the data on its second axis. Integers are interpreted as positional columns, while strings can reference DataFrame columns by name. A scalar string or int should be used where ``transformer`` expects X to be a 1d array-like (vector), otherwise a 2d array will be passed to the transformer. A callable is passed the input data `X` and can return any of the above. To select multiple columns by name or dtype, you can use :obj:`make_column_selector`.","[('domain_enc', ...)]"
,"remainder remainder: {'drop', 'passthrough'} or estimator, default='drop'By default, only the specified columns in `transformers` aretransformed and combined in the output, and the non-specifiedcolumns are dropped. (default of ``'drop'``).By specifying ``remainder='passthrough'``, all remaining columns thatwere not specified in `transformers`, but present in the data passedto `fit` will be automatically passed through. This subset of columnsis concatenated with the output of the transformers. For dataframes,extra columns not seen during `fit` will be excluded from the outputof `transform`.By setting ``remainder`` to be an estimator, the remainingnon-specified columns will use the ``remainder`` estimator. Theestimator must support :term:`fit` and :term:`transform`.Note that using this feature requires that the DataFrame columnsinput at :term:`fit` and :term:`transform` have identical order.",'passthrough'
,"sparse_threshold sparse_threshold: float, default=0.3If the output of the different transformers cont

In [ ]:
from sklearn.metrics import mean_absolute_error, root_mean_squared_error

preds = model.predict(X_test)

mae = mean_absolute_error(y_test, preds)
rmse = root_mean_squared_error(y_test, preds)
r2 = model.score(X_test, y_test)

print("MAE:", mae)
print("RMSE:", rmse)
print("R2:", r2)

MAE: 1636.9634375000003
RMSE: 1991.4495113661087
R2: -0.16100379093382178
